In [7]:
from collections import namedtuple

In [16]:
Point = namedtuple("Point", ['x', 'y'])

In [22]:
class FuzzyInput:
    def __init__(self, name, points, x_test):
        self.name = name
        self.points = points
        self.x_test = x_test
        self.mu = self.calcMu()
        
    def calcMu(self):
        if self.x_test <= self.points[0].x:
            return self.points[0].y
        elif self.x_test >= self.points[-1].x:
            return self.points[-1].y
        
        for p1, p2 in zip(self.points, self.points[1:]):
            if self.x_test >= p1.x and self.x_test <= p2.x:
                return FuzzyInput.line_through_two_points(p1, p2)(self.x_test)
        
    @staticmethod
    def line_through_two_points(a, b):
        c = (b.x - a.x) / (b.y - a.y)
        return lambda x_test: (x_test - a.x) / c + a.y

In [23]:
x = FuzzyInput('mala', [Point(3,1), Point(10,0)], 9)

In [27]:
x.mu

0.1428571428571429

In [28]:
from statistics import mean

In [37]:
class FuzzyOutput:
    def __init__(self, name, points):
        self.name = name
        self.points = points
        self.mu = 0
        self.c = self.calcC()
        
    def calcC(self):
        return mean([p.x for p in self.points if p.y == 1])
#         return mean(map(lambda p: p.x, filter(lambda p: p.y == 1, self.points)))

In [38]:
y = FuzzyOutput('asjaskl', [Point(7,0), Point(15,1), Point(25,1), Point(40,0)])

In [45]:
y.c

20

In [73]:
from enum import Enum, unique, auto

# @unique
class LogicOp(Enum):
    AND = auto()
    OR = auto()
    XOR = auto()

In [71]:
LogicOp.XOR

<LogicOp.XOR: 3>

In [49]:
LogicOp.AND

<LogicOp.AND: 0>

In [55]:
class Rule:
    def __init__(self, input1, input2, output, operator):
        if operator == LogicOp.AND:
            output.mu = max(output.mu, min(input1.mu, input2.mu))
        else:
            output.mu = max(output.mu, max(input1.mu, input2.mu))

In [56]:
potrosnja = []
potrosnja.append(FuzzyInput('mala', [Point(3,1), Point(10, 0)],9))
potrosnja.append(FuzzyInput('srednja', [Point(7,0), Point(10, 1), 
                                        Point(12,1), Point(15,0)],9))
potrosnja.append(FuzzyInput('velika', [Point(12,1), Point(15, 0)],9))

pouzdanost = []
pouzdanost.append(FuzzyInput('visoka', [Point(5,1),Point(10,0)], 8))
pouzdanost.append(FuzzyInput('niska', [Point(8,0),Point(15,1)], 8))

vrednost = []
vrednost.append(FuzzyOutput('mala', [Point(7,1),Point(15,0)]))
vrednost.append(FuzzyOutput('srednja', [Point(7,0),Point(15,1),
                                        Point(25,1), Point(40,0)]))
vrednost.append(FuzzyOutput('velika', [Point(25,0),Point(40,1)]))

rules = []
rules.append(Rule(potrosnja[0], pouzdanost[0], vrednost[2], LogicOp.AND))
rules.append(Rule(potrosnja[0], pouzdanost[1], vrednost[1], LogicOp.AND))
rules.append(Rule(potrosnja[1], pouzdanost[0], vrednost[1], LogicOp.AND))
rules.append(Rule(potrosnja[1], pouzdanost[1], vrednost[1], LogicOp.AND))
rules.append(Rule(potrosnja[2], pouzdanost[0], vrednost[1], LogicOp.AND))
rules.append(Rule(potrosnja[2], pouzdanost[1], vrednost[0], LogicOp.AND))


In [57]:
def defuzzify(ys):
    a = 0
    b = 0
    for y in ys:
        a += y.mu * y.c
        b += y.mu
    return a / b

In [58]:
defuzzify(vrednost)

25.263157894736842